In [1]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [4]:
# 연도별로 엑셀자료에 정렬해놓은 데이터 가져오기 (2010 ~ 2019년)
# 회사코드 Symbol에서 앞에 'A'제거하고 숫자만 가져오는 과정

data_2010 = pd.read_excel("코스피200 편입여부.xlsx", sheet_name="2010-01-31")
i = 0
for d in data_2010['Symbol']:
    data_2010.iloc[i, 0] = data_2010.iloc[i, 0][1:]
    i = i + 1

TypeError: 'float' object is not subscriptable

In [5]:
# company_code & company_name

K_2010 = data_2010.iloc[:200,0:2]
K_2010_code = list(K_2010["Symbol"])
K_2010_name = list(K_2010["company_name"])
K_2010_new = K_2010.set_index("Symbol")     # 'company_code'를 index로 만들어 제외된 회사, 추가된 회사를 list로 만들기 위한 변수

In [7]:
# Dart의 IP차단을 막기 위해 갯수 설정

K_2010_code_part = K_2010_code[:40]      # 40 ~ 80 , ... , 160 ~ 200
K_2010_code_part

['005930',
 '000660',
 '035420',
 '005380',
 '051910',
 '012330',
 '051900',
 '005490',
 '105560',
 '006400',
 '055550',
 '017670',
 '000270',
 '015760',
 '036570',
 '090430',
 '033780',
 '003550',
 '096770',
 '066570',
 '000810',
 '086790',
 '009150',
 '010950',
 '009540',
 '010130',
 '011170',
 '002790',
 '030200',
 '021240',
 '024110',
 '035250',
 '018880',
 '034220',
 '006800',
 '000720',
 '010140',
 '078930',
 '029780',
 '071050']

In [8]:
# 주소(url) setting

API_KEY = "9a9929b43fd4c79e06cc04d50eb34a70a0a576b1"
start_date = "20100101"
end_date = "20191231"
page_set = "100"   # 페이지당 건수
# auth : 발급받은 인증키(40자리)
# crp_cd : 공시대상회사의 종목코드(상장사:6자리 숫자) 또는 고유번호(기타법인:8자리 숫자)
# start_dt, end_dt : 검색시작,끝 접수일자
# dsn_tp : A001(사업보고서)

url_list = []
for crp in K_2010_code_part:
    company_code = crp   
    url = "http://dart.fss.or.kr/api/search.xml?auth=" + API_KEY + "&crp_cd=" + company_code + "&start_dt=" + start_date + "&end_dt=" + end_date + "&bsn_tp=A001&page_set=" + page_set  
    url_list.append(url)
url_list

['http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=005930&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=000660&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=035420&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=005380&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=051910&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=012330&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?a

In [9]:
# 각 연도별로 코스피200종목 회사들의 2010 ~ 2019년 모든 사업보고서 리스트를 데이터프레임으로 반환

def data_extract():
    url_lists = url_list
    data_finals = pd.DataFrame()
    for url in url_lists:
        xmlsoup = BeautifulSoup(urlopen(url).read(), 'html.parser')
        te = xmlsoup.findAll("list")
        data = pd.DataFrame()
        for t in te:
            temp = pd.DataFrame(([[t.crp_cls.string, t.crp_nm.string, t.crp_cd.string, t.rpt_nm.string,
                              t.rcp_no.string, t.flr_nm.string, t.rcp_dt.string, t.rmk.string]]),
                              columns = ["crp_cls", "crp_nm", "crp_cd", "rpt_nm", "rcp_no", "flr_nm", "rcp_dt", "rmk"])
            data = pd.concat([data,temp])
            data_result = data
        data_result = data_result[data_result['rmk'] != '정연']   # '정연'은 수정된 사업보고서가 제출되기 이전의 보고서이므로 제외
        data_finals = pd.concat([data_finals, data_result])
        data_finals = data_finals.reset_index(drop=True)
        # 중복된 사업보고서 삭제
        data_finals = data_finals.drop_duplicates(['crp_cls', 'crp_nm', 'crp_cd', 'rpt_nm', 'rcp_no', 'flr_nm', 'rcp_dt', 'rmk'], keep='first')
    return data_finals

In [10]:
report_result = data_extract()
report_result

,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,Y,삼성전자,005930,사업보고서 (2018.12),20190401004781,삼성전자,20190401,연
1,Y,삼성전자,005930,사업보고서 (2017.12),20180402005019,삼성전자,20180402,연
2,Y,삼성전자,005930,사업보고서 (2016.12),20170331004518,삼성전자,20170331,연
3,Y,삼성전자,005930,사업보고서 (2015.12),20160330003536,삼성전자,20160330,연
4,Y,삼성전자,005930,사업보고서 (2014.12),20150331002915,삼성전자,20150331,연
...,...,...,...,...,...,...,...,...
396,Y,한국금융지주,071050,사업보고서 (2013.12),20140331003336,한국금융지주,20140331,연
397,Y,한국금융지주,071050,사업보고서 (2013.03),20130628000501,한국금융지주,20130628,연
398,Y,한국금융지주,071050,사업보고서 (2012.03),20120629000615,한국금융지주,20120629,연
399,Y,한국금융지주,071050,사업보고서 (2011.03),20110629000241,한국금융지주,20110629,연


In [ ]:
# csv file로 저장

report_result.to_csv('C:/Users/PC/Desktop/2010/K_2010.csv', encoding='euc-kr') # 한글깨짐 방지

In [13]:
# filtering한 데이터 가져오기

data = pd.read_excel("C:/Users/PC/Desktop/2010/K_2010_2010.xlsx", sheet_name = 'Sheet1', encoding='euc-kr')
data = data.iloc[:-1,:]
data

,Unnamed: 0,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,8,Y,삼성전자,5930.0,[첨부추가]사업보고서 (2010.12),2.011033e+13,삼성전자,20110331.0,연
1,18,Y,SK하이닉스,660.0,사업보고서 (2010.12),2.011033e+13,SK하이닉스,20110331.0,연
2,28,Y,NAVER,35420.0,[첨부추가]사업보고서 (2010.12),2.011033e+13,NAVER,20110331.0,연
3,38,Y,현대자동차,5380.0,[첨부추가]사업보고서 (2010.12),2.011033e+13,현대자동차,20110331.0,연
4,48,Y,LG화학,51910.0,사업보고서 (2010.12),2.011033e+13,LG화학,20110331.0,연
5,58,Y,현대모비스,12330.0,[첨부추가]사업보고서 (2010.12),2.011033e+13,현대모비스,20110331.0,연
6,68,Y,LG생활건강,51900.0,사업보고서 (2010.12),2.011033e+13,LG생활건강,20110331.0,연
7,78,Y,포스코,5490.0,[기재정정]사업보고서 (2010.12),2.011041e+13,포스코,20110413.0,연
8,88,Y,KB금융,105560.0,사업보고서 (2010.12),2.011033e+13,KB금융,20110331.0,연
9,98,Y,삼성SDI,6400.0,[첨부추가]사업보고서 (2010.12),2.011033e+13,삼성SDI,20110331.0,연


In [14]:
# rcp_no로 해당 사업보고서 url과 name 저장

url_code = []
url_name = []
length = len(data)

for t in range(length):
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + str(data['rcp_no'][t])[:-2]
    url_code.append(url)
    name = data['crp_nm'][t]
    url_name.append(name)

In [15]:
url_code

['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331002193',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331003040',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331001371',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331002474',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331002957',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331003027',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331002226',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110413001032',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331002162',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331002630',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20121026000253',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331002031',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331002861',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331002045',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331001532',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=201103310

In [16]:
url_name

['삼성전자',
 'SK하이닉스',
 'NAVER',
 '현대자동차',
 'LG화학',
 '현대모비스',
 'LG생활건강',
 '포스코',
 'KB금융',
 '삼성SDI',
 '신한지주',
 'SK텔레콤',
 '기아자동차',
 '한국전력공사',
 '엔씨소프트',
 '아모레퍼시픽',
 '케이티앤지',
 'LG',
 'SK이노베이션',
 'LG전자',
 '삼성화재해상보험',
 '하나금융지주',
 '삼성전기',
 'S-Oil',
 '고려아연',
 '롯데케미칼',
 '아모레퍼시픽그룹',
 '케이티',
 '웅진코웨이',
 '기업은행',
 '강원랜드',
 '한온시스템',
 'LG디스플레이',
 '미래에셋대우',
 '현대건설',
 '삼성중공업',
 'GS',
 '삼성카드',
 '한국금융지주']

In [17]:
# Dart 의 IP차단을 막기 위해 갯수 설정
part_code = url_code[:20]
part_name = url_name[:20]

In [18]:
# 사업보고서 url에 그대로 들어가면 각 파트별 HTML코드가 없음.
# 따라서, 각 사업보고서의 HTML 코드에서 '이사의 경영진단 및 분석의견' url을 구한 뒤
# BeautifulSoup으로 HTML코드 partitioning

page_list = []
for code in part_code:
    page = BeautifulSoup(urlopen(code).read(), 'html.parser')  
    body = str(page.find('head')).split('이사의 경영진단 및 분석의견",')[1].split('cnt++')[0].split('viewDoc(')[1].split(')')[0].split(', ')
    body = [body[i][1:-1] for i in range(len(body))]   
    url_final = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=' + body[0] + '&dcmNo=' + body[1] + '&eleId=' + body[2] + '&offset=' + body[3] + '&length=' + body[4] + '&dtd=' + body[5]
    page = BeautifulSoup(urlopen(url_final).read(), 'html.parser')
    page_list.append(page)
    print(url_final)
page_list

http://dart.fss.or.kr/report/viewer.do?rcpNo=20110331002193&dcmNo=2965731&eleId=13&offset=427941&length=39397&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20110331003040&dcmNo=2970028&eleId=13&offset=454654&length=19774&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20110331001371&dcmNo=2961515&eleId=13&offset=219589&length=11243&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20110331002474&dcmNo=2967188&eleId=13&offset=559916&length=16776&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20110331002957&dcmNo=2969558&eleId=13&offset=416925&length=71284&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20110331003027&dcmNo=2969968&eleId=13&offset=367753&length=22867&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20110331002226&dcmNo=2965899&eleId=13&offset=796113&length=14263&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20110413001032&dcmNo=3003683&eleId=15&offset=454595&length=54622&dtd=dart3.xsd
http://d

[<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
 
 <html style="border:0">
 <head>
 <title></title>
 <meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible">
 <meta content="gdi" http-equiv="X-UA-TextLayoutMetrics">
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
 <link href="/css/report_xml.css" rel="stylesheet" type="text/css">
 </link></meta></meta></meta></head>
 <body bgcolor="#FFFFFF">
 <p><br/></p>
 <p class="section-1"><a name="toc1">V. 이사의 경영진단 및 분석의견</a></p>
 <p><br/></p>
 <p><span style="color:#FF0000;"></span><span style="font-size:16pt;line-height:1.6em;font-weight:bold;">1. 예측정보에 대한 주의사항<br/></span><span><br/>본 자료는 미래에 대한 "예측정보"를 포함하고 있습니다.<br/>이는 과거가 아닌 미래의 사건과 관계된 것으로 회사의 향후 예상되는 경영현황 및 재무실적을 의미하고, 표현상으로는 '예상', '전망', '계획', '기대' 등과 같은 단어를 포함합니다.<br/></span><span><br/></span><span>"예측정보"는 그 성격상 불확실한 사건들을 언급하는데, 회사의 향후 경영현황 및 재무실적에 긍정적 또는 부정적으로 영향을 미칠 수 있는 불확실성에는 다음과 같은 것들이 포함됩니다.<br/></span><s

In [22]:
import re
def remove_tag(content):             ## 태그를 제외하고 text만 가져오는 함수
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '\n', content)
    return cleantext
def remove_space(content):          # 개행문자인 \n과 \xa0제거
    cleans = re.compile('[\n]|[\xa0]')
    cleantext = re.sub(cleans, '', content)
    return cleantext

In [23]:
# partitioning한 코드에서 재무제표 등의 표를 제거하고 텍스트만 가져오기 위해 'table'태그를 제거하는 과정
# 표를 제거하기 위해 표를 크롤링해오는 함수를 만들어보고, 정규표현식을 써보고, HTML코드의 속성값으로 텍스트를 가져와보는 등 실패

page_partition = []
for page_char in page_list:
    whole_data = str(page_char).split('<')
    table_data = page_char.find_all('table')
    table_data_li = str(table_data).split('<')
    whole_data_copy = whole_data.copy()

    for table in table_data_li:
        if (table in whole_data) == True:
            whole_data_copy.remove(table)
        
    output = '<'.join(whole_data_copy)

    output1 = remove_tag(output)
    final = remove_space(output1)
    page_partition.append(final)

In [24]:
page_partition

['V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 미래에 대한 "예측정보"를 포함하고 있습니다.이는 과거가 아닌 미래의 사건과 관계된 것으로 회사의 향후 예상되는 경영현황 및 재무실적을 의미하고, 표현상으로는 \'예상\', \'전망\', \'계획\', \'기대\' 등과 같은 단어를 포함합니다."예측정보"는 그 성격상 불확실한 사건들을 언급하는데, 회사의 향후 경영현황 및 재무실적에 긍정적 또는 부정적으로 영향을 미칠 수 있는 불확실성에는 다음과 같은 것들이 포함됩니다.·환율, 이자율 등의 변동을 포함한 국내·외 금융 시장의 동향·사업의 처분, 매수 등을 포함한 회사의 전략적인 의사결정·반도체, LCD, 정보통신, 디지털미디어 등 회사가 영위하는 주요사업 분야의  예상치 못한 급격한 여건변화 ·기타 경영현황 및 재무실적에 영향을 미칠 수 있는 국내·외적 변화이러한 불확실성으로 인해 회사의 실제 미래실적은 "예측정보"에 명시적 또는 묵시적으로 포함된 내용과 중대한 차이가 있을 수 있음을 양지하시기 바랍니다.당사는 동 예측정보 작성시점 이후에 발생하는 위험 또는 불확실성을 반영하기 위하여 예측정보에 기재한 사항을 수정하는 정정보고서를 공시할 의무는 없습니다.2. 개요2010년 당사는 선진국 경기회복 지연, 유럽 재정위기, 환율변동성 확대 등 어려운 여건에서도 사상 최대 매출 및 영업이익을 달성하였습니다. 특히, 메모리, TV, HHP 등주력 사업에서 시장지배력을 확대하는 한편, 제품 차별화를 통하여 시장경쟁의 심화에도 불구하고 견고한 이익을 달성하였습니다. TV 부문은 시장경쟁 심화에 따른 판가하락에도 불구하고, 3D/Smart TV 출시를 통해 프리미엄 시장을 장악하였고, 경쟁사에 비해 앞선 제품력과 디자인으로 2006년이후 지속 1위를 유지하였습니다. 휴대폰 부문은 Galaxy S가 판매량 10백만대를 돌파하며 스마트폰의 폭발적 성장을 견인하였고, 서남아ㆍ중남미등 신흥시장에서의 고성장으로 세계시장 점유율 20%를 돌파하여, 사업위상을 지

In [ ]:
# txt file로 저장

result_texts = zip(part_name, page_partition)
for name,text in result_texts:
    file = open('C:/Users/PC/Desktop/2010_text/%s 사업보고서.txt' %name, 'w', encoding='utf-8')
    file.write(text)
    file.close()